In [2]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops

class GatedGNNLayer(MessagePassing):
    def __init__(self, in_channels: int, out_channels: int):
        super(GatedGNNLayer, self).__init__(aggr='add')
        self.message_nn = nn.Linear(out_channels, out_channels)
        self.update_nn = nn.GRUCell(out_channels, out_channels)

    def forward(self, x, edge_index, h):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        return self.propagate(edge_index, x=x, h=h)

    def message(self, h_j, x_i):
        return self.message_nn(h_j)

    def update(self, aggr_out, h):
        return self.update_nn(aggr_out, h)

class GGNN(nn.Module):
    def __init__(self, in_channels, out_channels, n_steps):
        super(GGNN, self).__init__()
        self.n_steps = n_steps
        self.ggnn_layer = GatedGNNLayer(in_channels, out_channels)
        self.h_init = nn.Linear(in_channels, out_channels)


    def forward(self, x, edge_index):
        h = self.h_init(x)
        for _ in range(self.n_steps):
            h = self.ggnn_layer(x, edge_index, h)
        return h

num_features = 10
num_classes = 2
num_nodes = 7
n_steps = 3

x = torch.randn(num_nodes, num_features)
edge_index = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 0], [1, 0, 3, 2, 5, 4, 0, 6]])

model = GGNN(num_features, num_classes, n_steps)
output = model(x, edge_index)
print(output)


tensor([[-0.1613,  0.6986],
        [ 0.0789,  0.7622],
        [-0.1312,  0.6711],
        [-0.0755,  0.7066],
        [-0.2191,  0.6435],
        [-0.2056,  0.6470],
        [-0.2087,  0.6525]], grad_fn=<AddBackward0>)


In [4]:
import torch
from torch_geometric.data import Data

class GGNN(nn.Module):
    def __init__(self, in_channels, out_channels, n_steps):
        super(GGNN, self).__init__()
        self.n_steps = n_steps
        self.ggnn_layer = GatedGNNLayer(in_channels, out_channels)
        self.h_init = nn.Linear(in_channels, out_channels)
        self.classifier = nn.Linear(out_channels, 2)


    def forward(self, x, edge_index):
        h = self.h_init(x)  # Initialize hidden state
        for _ in range(self.n_steps):
            h = self.ggnn_layer(x, edge_index, h)
        return torch.softmax(self.classifier(h), dim=1)

node_features = torch.arange(1, 11).view(-1, 1).float()
node_labels = torch.tensor([1]*5 + [0]*5)

# Fully connected graph
edge_index = torch.combinations(torch.arange(10), r=2).t().contiguous()

# Graph Data
data = Data(x=node_features, edge_index=edge_index, y=node_labels)
data

Data(x=[10, 1], edge_index=[2, 45], y=[10])

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.7505108118057251
Epoch 1, Loss: 0.7345073223114014
Epoch 2, Loss: 0.7209871411323547
Epoch 3, Loss: 0.710412323474884
Epoch 4, Loss: 0.7028274536132812
Epoch 5, Loss: 0.6978403329849243
Epoch 6, Loss: 0.6947283744812012
Epoch 7, Loss: 0.6926599740982056
Epoch 8, Loss: 0.6908921003341675
Epoch 9, Loss: 0.6888076066970825
Epoch 10, Loss: 0.6857887506484985
Epoch 11, Loss: 0.681104838848114
Epoch 12, Loss: 0.6741296052932739
Epoch 13, Loss: 0.6648052930831909
Epoch 14, Loss: 0.6538203954696655
Epoch 15, Loss: 0.6421905755996704
Epoch 16, Loss: 0.6306681632995605
Epoch 17, Loss: 0.6196225881576538
Epoch 18, Loss: 0.6091436147689819
Epoch 19, Loss: 0.5991804003715515
Epoch 20, Loss: 0.5896567702293396
Epoch 21, Loss: 0.5805214643478394
Epoch 22, Loss: 0.5717483162879944
Epoch 23, Loss: 0.5633229613304138
Epoch 24, Loss: 0.5552337169647217
Epoch 25, Loss: 0.5474665760993958
Epoch 26, Loss: 0.5400010347366333
Epoch 27, Loss: 0.5328039526939392
Epoch 28, Loss: 0.52582538127899

In [6]:
with torch.no_grad():
    pred = model(data.x, data.edge_index)
    predicted_labels = pred.max(1)[1]
    print(predicted_labels)

tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])


In [7]:
import torch
from torch_geometric.data import Data

node_features = torch.arange(1, 11).view(-1, 1).float()
node_labels = torch.tensor([1]*5 + [0]*5)

mask = torch.randperm(10)[:3]
train_mask = torch.ones(10, dtype=torch.bool)
train_mask[mask] = False

edge_index = torch.combinations(torch.arange(10), r=2).t().contiguous()

data = Data(x=node_features, edge_index=edge_index, y=node_labels, train_mask=train_mask)
data.validate(raise_on_error=True)
data

Data(x=[10, 1], edge_index=[2, 45], y=[10], train_mask=[10])

In [8]:
model = GGNN(1, num_classes, n_steps)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)

    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.7045034766197205
Epoch 1, Loss: 0.6975182890892029
Epoch 2, Loss: 0.6906991004943848
Epoch 3, Loss: 0.6840797066688538
Epoch 4, Loss: 0.6778831481933594
Epoch 5, Loss: 0.6722598075866699
Epoch 6, Loss: 0.667226254940033
Epoch 7, Loss: 0.6626810431480408
Epoch 8, Loss: 0.6584886908531189
Epoch 9, Loss: 0.6545923948287964
Epoch 10, Loss: 0.6509963870048523
Epoch 11, Loss: 0.6472967863082886
Epoch 12, Loss: 0.6428599953651428
Epoch 13, Loss: 0.6377127766609192
Epoch 14, Loss: 0.6321994066238403
Epoch 15, Loss: 0.6263987421989441
Epoch 16, Loss: 0.6200027465820312
Epoch 17, Loss: 0.6127097010612488
Epoch 18, Loss: 0.6045129895210266
Epoch 19, Loss: 0.5957077741622925
Epoch 20, Loss: 0.5865606069564819
Epoch 21, Loss: 0.5766382813453674
Epoch 22, Loss: 0.5658078789710999
Epoch 23, Loss: 0.5547047257423401
Epoch 24, Loss: 0.5432830452919006
Epoch 25, Loss: 0.5311815142631531
Epoch 26, Loss: 0.5188978910446167
Epoch 27, Loss: 0.506672739982605
Epoch 28, Loss: 0.49388226866722

In [9]:
with torch.no_grad():
    pred = model(data.x, data.edge_index)
    predicted_labels = pred.max(1)[1]

    predictions_for_unknown = predicted_labels[~data.train_mask]
    print("Predictions for unknown nodes:", predictions_for_unknown)

Predictions for unknown nodes: tensor([1, 0, 0])


In [10]:
import torch
from torch_geometric.data import Data
import numpy as np

def create_graph(node_values):
    num_nodes = len(node_values)
    node_features = torch.tensor(node_values, dtype=torch.float).view(-1, 1)
    node_labels = torch.tensor([1 if value < 50 else 0 for value in node_values], dtype=torch.long)

    edge_index = torch.tensor(np.random.choice(num_nodes, (2, num_nodes * 2)), dtype=torch.long)

    return Data(x=node_features, edge_index=edge_index, y=node_labels)

def create_graph_v2(node_values):
    num_nodes = len(node_values)
    node_features = torch.tensor(node_values, dtype=torch.float).view(-1, 1)


    edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if node_values[i] >= node_values[j]], dtype=torch.long).t().contiguous()
    
    node_labels = torch.tensor([1 if len(np.where(edge_index[0] == i)[0]) > 25 else 0 for i in range(num_nodes)], dtype=torch.long)
    

    return Data(x=node_features, edge_index=edge_index, y=node_labels)

train_values = np.random.choice(range(1, 100), 50, replace=False)
test_values = np.random.choice(range(1, 100), 50, replace=False)

train_data = create_graph_v2(train_values)
test_data = create_graph_v2(test_values)

train_data

Data(x=[50, 1], edge_index=[2, 1275], y=[50])

In [11]:
print("Training Data - Features:", train_data.x[:5])
print("Training Data - Labels:", train_data.y[:5])
print("Testing Data - Features:", test_data.x[:5])
print("Testing Data - Labels:", test_data.y[:5])

Training Data - Features: tensor([[42.],
        [45.],
        [98.],
        [10.],
        [19.]])
Training Data - Labels: tensor([0, 0, 1, 0, 0])
Testing Data - Features: tensor([[76.],
        [16.],
        [39.],
        [56.],
        [54.]])
Testing Data - Labels: tensor([1, 0, 0, 1, 1])


In [12]:
model = GGNN(1, num_classes, n_steps)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()
    out = model(train_data.x, train_data.edge_index)
    loss = criterion(out, train_data.y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.6945098042488098
Epoch 1, Loss: 0.6083464026451111
Epoch 2, Loss: 0.7125699520111084
Epoch 3, Loss: 0.6801719665527344
Epoch 4, Loss: 0.5837964415550232
Epoch 5, Loss: 0.573807954788208
Epoch 6, Loss: 0.6017743349075317
Epoch 7, Loss: 0.602378785610199
Epoch 8, Loss: 0.60025954246521
Epoch 9, Loss: 0.5961257815361023
Epoch 10, Loss: 0.5903216600418091
Epoch 11, Loss: 0.5836807489395142
Epoch 12, Loss: 0.5762725472450256
Epoch 13, Loss: 0.5689895153045654
Epoch 14, Loss: 0.5619006156921387
Epoch 15, Loss: 0.555547297000885
Epoch 16, Loss: 0.5503217577934265
Epoch 17, Loss: 0.5465654134750366
Epoch 18, Loss: 0.5443823337554932
Epoch 19, Loss: 0.5428814888000488
Epoch 20, Loss: 0.5416820645332336
Epoch 21, Loss: 0.5403760671615601
Epoch 22, Loss: 0.5385956764221191
Epoch 23, Loss: 0.5361430644989014
Epoch 24, Loss: 0.5330132842063904
Epoch 25, Loss: 0.5293608903884888
Epoch 26, Loss: 0.5254391431808472
Epoch 27, Loss: 0.5215299129486084
Epoch 28, Loss: 0.5178781747817993


In [13]:
with torch.no_grad():
    test_out = model(test_data.x, test_data.edge_index)
    predicted_labels = test_out.max(1)[1]
    accuracy = (predicted_labels == test_data.y).sum().item() / test_data.y.size(0)
    print("Test Accuracy:", accuracy)

Test Accuracy: 0.96
